In [79]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK, lpSum

In [113]:
# Creates dictionaries of products and their materials, labor, selling price, advertising costs, etc
periods = list(range(1,6))
products = ['widgets', 'gadgets', 'flugels']
ad_costs_list = [160, 120, 180]
product_selling_price_list = [2490, 1990, 2970]
plant_a_rm_1_requirements_list = [194, 230, 178]
plant_a_rm_2_requirements_list = [8.6, 0, 11.6]
plant_b_rm_1_requirements_list = [188, 225, 170]
plant_b_rm_2_requirements_list = [9.2, 0, 10.8]
plant_a_labor_requirements_list = [9.5, 7.1, 11.1]
plant_b_labor_requirements_list = [9.1, 7.8, 10.6]
plant_a_inventory_costs_list = [7.5, 5.5, 6.5]
plant_b_inventory_costs_list = [7.8, 5.7, 7.0]

ad_costs = dict(zip(products, ad_costs_list))
product_selling_prices = dict(zip(products, product_selling_price_list))
plant_a_rm_1_requirements = dict(zip(products, plant_a_rm_1_requirements_list))
plant_a_rm_2_requirements = dict(zip(products, plant_a_rm_2_requirements_list))
plant_b_rm_1_requirements = dict(zip(products, plant_b_rm_1_requirements_list))
plant_b_rm_2_requirements = dict(zip(products, plant_b_rm_2_requirements_list))
plant_a_labor_requirements = dict(zip(products, plant_a_labor_requirements_list))
plant_b_labor_requirements = dict(zip(products, plant_b_labor_requirements_list))
plant_a_inventory_costs = dict(zip(products, plant_a_inventory_costs_list))
plant_b_inventory_costs = dict(zip(products, plant_b_inventory_costs_list))

In [114]:
# define variables
ads_variables = []
production_variables = []
demand_from_contracts_variables = []
demand_from_ads_variables = []
raw_material_1_to_buy_for_plant_a_variables = []
raw_material_1_to_buy_for_plant_b_variables = []
raw_material_2_to_buy_for_plant_a_variables = []
raw_material_2_to_buy_for_plant_b_variables = []
inventory_plant_a = []
inventory_plant_b = []
labor_variables_in_plant_a = []
labor_variables_in_plant_b = []
overtime_variables_in_plant_a = []
overtime_variables_in_plant_b = []

for period in periods:
    raw_material_1_to_buy_for_plant_a_variables.append(f"rm_1_for_plant_a_in_p{period}")
    raw_material_1_to_buy_for_plant_b_variables.append(f"rm_1_for_plant_b_in_p{period}")
    raw_material_2_to_buy_for_plant_a_variables.append(f"rm_2_for_plant_a_in_p{period}")
    raw_material_2_to_buy_for_plant_b_variables.append(f"rm_2_for_plant_b_in_p{period}")
    labor_variables_in_plant_a.append(f"labor_for_plant_a_in_p{period}")
    labor_variables_in_plant_b.append(f"labor_for_plant_b_in_p{period}")
    overtime_variables_in_plant_a.append(f"overtime_labor_for_plant_a_in_p{period}")
    overtime_variables_in_plant_b.append(f"overtime_labor_for_plant_b_in_p{period}")
    for product in products:
        demand_from_contracts_variables.append(f"{product}_contracted_demand_in_p{period}")
        production_variables.append(f"{product}_produced_in_plant_a_in_p{period}")
        production_variables.append(f"{product}_produced_in_plant_b_in_p{period}")
        inventory_plant_a.append(f"{product}_inventoried_in_plant_a_in_p{period}")
        inventory_plant_b.append(f"{product}_inventoried_in_plant_b_in_p{period}")
        if period != 5:
            ads_variables.append(f"{product}_advertising_investment_p{period}")
        if period != 1:
            demand_from_ads_variables.append(f"{product}_demand_from_ads_p{period}")

In [115]:
# Create dictionaries to store the referenced Variables
advertising_vars = LpVariable.dicts("", ads_variables, 0)
production_variables = LpVariable.dicts("", production_variables, 0)
demand_from_contracts_vars = LpVariable.dicts("", demand_from_contracts_variables, 0)
demand_from_ads_vars = LpVariable.dicts("", demand_from_ads_variables, 0)
inventory_plant_a_vars = LpVariable.dicts("", inventory_plant_a, 0)
inventory_plant_b_vars = LpVariable.dicts("", inventory_plant_b, 0)
labor_in_plant_a_vars = LpVariable.dicts("", labor_variables_in_plant_a, 0)
labor_in_plant_b_vars = LpVariable.dicts("", labor_variables_in_plant_b, 0)
overtime_labor_in_plant_a_vars = LpVariable.dicts("", overtime_variables_in_plant_a, 0)
overtime_labor_in_plant_b_vars = LpVariable.dicts("", overtime_variables_in_plant_b, 0)
rm_1_to_buy_for_plant_a_vars = LpVariable.dicts("", raw_material_1_to_buy_for_plant_a_variables, 0)
rm_1_to_buy_for_plant_b_vars = LpVariable.dicts("", raw_material_1_to_buy_for_plant_b_variables, 0)
rm_2_to_buy_for_plant_a_vars = LpVariable.dicts("", raw_material_2_to_buy_for_plant_a_variables, 0)
rm_2_to_buy_for_plant_b_vars = LpVariable.dicts("", raw_material_2_to_buy_for_plant_b_variables, 0)
    
    
# defines the problem
nu_industries = LpProblem("NU_Industries", LpMaximize)

## Objective

Throughout the planning horizon, NU Industries will sell: 
* Widgets for \\$2490 
* Gadgets for \\$1990 
* Flugels for \\$2970. 

The products can be manufactured at either of NU's Manufacturing Plants.


In [122]:
## Objective
nu_industries += (
    ## This is demand from ads times the product selling price
    lpSum([product_selling_prices[product] * demand_from_ads_vars[f'{product}_demand_from_ads_p{period}'] for product 
           in products for period in periods if period != 1]) +  
    ## This is contracted demand
    lpSum([product_selling_prices[product] * demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}'] for product 
           in products for period in periods]) -
    ## This is the cost of ads
    lpSum([ad_costs[product] * advertising_vars[f'{product}_advertising_investment_p{period}'] for product 
           in products for period in periods if period != 5]) - 
    
    ## This is the cost of regular-time labor in a - Labor costs during periods 1 and 2 are $11/hour for regular time 
    lpSum([labor_in_plant_a_vars[f"labor_for_plant_a_in_p{period}"] * 11
           for period in periods if period == 1 or period == 2]) - 
    ## This is the cost of regular-time labor in b - Labor costs during periods 1 and 2 are $11/hour for regular time 
    lpSum([labor_in_plant_b_vars[f"labor_for_plant_b_in_p{period}"] * 11
           for period in periods if period == 1 or period == 2])  -
    ## Labor costs are expected to rise by 5% in plant a at the end of period 2
    lpSum([labor_in_plant_a_vars[f"labor_for_plant_a_in_p{period}"] * (11 * 1.05)
           for period in periods if period > 2]) -
    ## Labor costs are expected to rise by 10% in plant a at the end of period 2
    lpSum([labor_in_plant_b_vars[f"labor_for_plant_b_in_p{period}"] * (11 * 1.1)
           for period in periods if period > 2]) - 
    
    ## $16.50/hour for overtime. 
    lpSum([overtime_labor_in_plant_a_vars[f"overtime_labor_for_plant_a_in_p{period}"] * 16.5
           for period in periods if period == 1 or period == 2]) - 
    ## This is the cost of regular-time labor in b - Labor costs during periods 1 and 2 are $11/hour for regular time 
    lpSum([overtime_labor_in_plant_b_vars[f"overtime_labor_for_plant_b_in_p{period}"] * 16.5
           for period in periods if period == 1 or period == 2])  -
    ## Labor costs are expected to rise by 5% in plant a at the end of period 2
    lpSum([overtime_labor_in_plant_a_vars[f"overtime_labor_for_plant_a_in_p{period}"] * (16.5 * 1.05)
           for period in periods if period > 2]) -
    ## Labor costs are expected to rise by 10% in plant a at the end of period 2
    lpSum([overtime_labor_in_plant_b_vars[f"overtime_labor_for_plant_b_in_p{period}"] * (16.5 * 1.1)
           for period in periods if period > 2]) -
    
    ## Inventory in A and B
    #widgets_inventoried_in_plant_a_in_p1
    lpSum([inventory_plant_a_vars[f"{product}_inventoried_in_plant_a_in_p{period}"] * plant_a_inventory_costs[product]
           for product in products for period in periods]) - 
    lpSum([inventory_plant_b_vars[f"{product}_inventoried_in_plant_b_in_p{period}"] * plant_b_inventory_costs[product]
           for product in products for period in periods])
    
    ,'Maximize_Profits'
)

/Users/robords/.venv/msds460_final/lib/python3.9/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


A

The product inventory area can store a combined maximum of 70 units, and inventory costs are as follows. It costs \\$7.50 to store one Widget from one period to the next. The corresponding costs for Gadgets and Flugels are \\$5.50 and \\$6.50, respectively.

B

The product inventory area can store a combined maximum of 50 units, and inventory costs are as follows. It costs \\$7.80 to store one Widget from one period to the next. The corresponding costs for Gadgets and Flugels are \\$5.70 and \\$7.00, respectively.

# Define Constraints

### Contractual Production Requirements:    
This is demand in a given period that we are already contracted to sell

|Product|Period 1|Period 2|Period 3|Period 4|Period 5|
|-------|--------|--------|--------|--------|--------|
|Widgets|70      |125     |185     |190     |200     |
|Gadgets|200     |300     |295     |245     |240     |
|Flugels|140     |175     |205     |235     |230     |

In [123]:
# Define Production Requirement Dictionary
production_contractual_requirements_list = [70, 200, 140, 125, 300, 175, 185, 295, 205, 190, 245, 235, 200, 240, 230]
production_contractual_requirements = dict(zip([f"p_{period}_{product}_requirement" for period 
                                                in periods for product in products], 
                                               production_contractual_requirements_list))


In [124]:
# Production Requirements - demand must exceed our contracted production requirements
for period in periods:
    for product in products:
        nu_industries += (demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}']
                          >= production_contractual_requirements[f'p_{period}_{product}_requirement'], 
                          f"{product}_required_in_p{period}")


### Advertising

Marketing & Forecasting Department anticipates that NU Industries can cultivate additional demand within the distribution area through effective advertising. The M&F Department projects that each \\$160 invested in advertising Widgets in a particular period creates additional demand for one Widget in the next period. The corresponding values for Gadgets and Flugels are \\$120 and \\$180, respectively. However, the total advertising budget is limited to \\$70,000 for the entire planning horizon.

In [125]:
# Advertising Budget
nu_industries += (lpSum([advertising_vars[f'{product}_advertising_investment_p{period}'] for product 
                         in products for period in periods if period != 5]) <= 70000,
                  "Advertising_Budget")

In [126]:
# Additional Demand from Advertising
for period in periods:
    for product in products:
        if period == 1:
            pass
        else:
            nu_industries += (demand_from_ads_vars[f'{product}_demand_from_ads_p{period}'] * ad_costs[product] == 
                              advertising_vars[f'{product}_advertising_investment_p{period-1}'],
                              f"{product}_Demand_from_Ads_in_p{period}")

### Production Requirements

Demand must be less than or equal to inventory from the previous period + production in that same period


In [127]:
for period in periods:
    for product in products:
        if period == 1:
            # no inventory in this period, and no demand from ads
            nu_industries += (demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}'] <=
                              production_variables[f"{product}_produced_in_plant_a_in_p{period}"] + 
                              production_variables[f"{product}_produced_in_plant_b_in_p{period}"],
                             f"{product}_production_requirements_in_p{period}")
        else:
            nu_industries += (demand_from_ads_vars[f'{product}_demand_from_ads_p{period}'] + 
                              demand_from_contracts_vars[f'{product}_contracted_demand_in_p{period}'] <=
                              production_variables[f"{product}_produced_in_plant_a_in_p{period}"] + 
                              production_variables[f"{product}_produced_in_plant_b_in_p{period}"] + 
                              inventory_plant_a_vars[f"{product}_inventoried_in_plant_a_in_p{period - 1}"] + 
                              inventory_plant_b_vars[f"{product}_inventoried_in_plant_b_in_p{period - 1}"],
                              f"{product}_production_requirements_in_p{period}")
                              
        

```python
plant_a_rm_1_requirements = dict(zip(products, plant_a_rm_1_requirements_list))
plant_a_rm_2_requirements = dict(zip(products, plant_a_rm_2_requirements_list))
plant_b_rm_1_requirements = dict(zip(products, plant_b_rm_1_requirements_list))
plant_b_rm_2_requirements = dict(zip(products, plant_b_rm_2_requirements_list))
plant_a_labor_requirements = dict(zip(products, plant_a_labor_requirements_list))
plant_b_labor_requirements = dict(zip(products, plant_b_labor_requirements_list))
```

A
* Each Widget requires 194 pounds of Raw Material 1, 8.6 pounds of Raw Material 2, and 9.5 hours of Labor.
* Each Gadget requires 230 pounds of Raw Material 1 and 7.1 hours of Labor.
* Each Flugel requires 178 pounds of Raw Material 1, 11.6 pounds of Raw Material 2, and 11.1 hours of Labor.

B
* Each Widget requires 188 pounds of Raw Material 1, 9.2 pounds of Raw Material 2, and 9.1 hours of Labor.
* Each Gadget requires 225 pounds of Raw Material 1 and 7.8 hours of Labor.
* Each Flugel requires 170 pounds of Raw Material 1, 10.8 pounds of Raw Material 2, and 10.6 hours of Labor.

In [128]:
# raw material consumption must be less than or equal to the production in a given period
for period in periods:
    ## A maximum of 70 tons of Raw Material 1 are available from a raw material vendor each period.
    nu_industries += (lpSum([(production_variables[f"{product}_produced_in_plant_a_in_p{period}"] * 
                       plant_a_rm_1_requirements[product]) for product in products]) + 
                       lpSum([(production_variables[f"{product}_produced_in_plant_b_in_p{period}"] * 
                       plant_b_rm_1_requirements[product]) for product in products]) <= 70 * 2000,
                      f"Tons_of_Raw_Material_1_Available_Period_{period}")
    ## A maximum of 2.5 tons of Raw Material 1 are available from a raw material vendor each period.        
    nu_industries += (lpSum([(production_variables[f"{product}_produced_in_plant_a_in_p{period}"] * 
               plant_a_rm_2_requirements[product]) for product in products]) + 
               lpSum([(production_variables[f"{product}_produced_in_plant_b_in_p{period}"] * 
               plant_b_rm_2_requirements[product]) for product in products]) <= 2.5 * 2000,
              f"Tons_of_Raw_Material_2_Available_Period_{period}")

### Labor Requirements:

__A__    
Regular time labor availability is limited to 2500 hours in each period, but overtime can be scheduled in any amount if necessary. 

__B__    
Regular time labor availability is limited to 3800 hours in each period, but overtime can be scheduled in any amount if necessary. 

A
* Each Widget requires 9.5 hours of Labor.
* Each Gadget requires 7.1 hours of Labor.
* Each Flugel requires 11.1 hours of Labor.

B
* Each Widget requires 9.1 hours of Labor.
* Each Gadget requires 7.8 hours of Labor.
* Each Flugel requires 10.6 hours of Labor.

In [129]:
for period in periods:
    nu_industries += (lpSum([(production_variables[f"{product}_produced_in_plant_a_in_p{period}"] * 
                              plant_a_labor_requirements[product]) for product in products]) - 
                      overtime_labor_in_plant_a_vars[f"overtime_labor_for_plant_a_in_p{period}"] <= 2500,
                      f"Plant_A_Labor_Available_Period_{period}")
    
    nu_industries += (lpSum([(production_variables[f"{product}_produced_in_plant_b_in_p{period}"] * 
                          plant_a_labor_requirements[product]) for product in products]) - 
                  overtime_labor_in_plant_b_vars[f"overtime_labor_for_plant_b_in_p{period}"] <= 3800,
                  f"Plant_B_Labor_Available_Period_{period}")
        


## Inventory

A

The product inventory area can store a combined maximum of 70 units, and inventory costs are as follows. It costs \\$7.50 to store one Widget from one period to the next. The corresponding costs for Gadgets and Flugels are \\$5.50 and \\$6.50, respectively.

B

The product inventory area can store a combined maximum of 50 units, and inventory costs are as follows. It costs \\$7.80 to store one Widget from one period to the next. The corresponding costs for Gadgets and Flugels are \\$5.70 and \\$7.00, respectively.

In [130]:
for period in periods:
    nu_industries += (lpSum([(inventory_plant_a_vars[f"{product}_inventoried_in_plant_a_in_p{period}"]) 
                             for product in products]) <= 70,
                      f"Plant_A_Inventory_Cap_{period}")
    
    nu_industries += (lpSum([(inventory_plant_b_vars[f"{product}_inventoried_in_plant_b_in_p{period}"]) 
                             for product in products]) <= 50,
                      f"Plant_B_Inventory_Cap_{period}")

## Transportation

There is an average transportation cost associated with the shipment of each unit of finished product from each plant to the Distribution Center. The cost depends on the product and the plant of origin. The unit shipping costs are given in the following table.

|Product |	Plant A |	Plant B |
|-------|--------|--------|
|Widgets |	\\$6.30 | 	\\$6.50 | 
|Gadgets |	\\$4.60 | 	\\$5.00 |
|Flugels |	\\$5.50 | 	\\$5.70 |

In [131]:
# The problem data is written to an .lp file
nu_industries.writeLP("NU_Industries.lp")

[_flugels_advertising_investment_p1,
 _flugels_advertising_investment_p2,
 _flugels_advertising_investment_p3,
 _flugels_advertising_investment_p4,
 _flugels_contracted_demand_in_p1,
 _flugels_contracted_demand_in_p2,
 _flugels_contracted_demand_in_p3,
 _flugels_contracted_demand_in_p4,
 _flugels_contracted_demand_in_p5,
 _flugels_demand_from_ads_p2,
 _flugels_demand_from_ads_p3,
 _flugels_demand_from_ads_p4,
 _flugels_demand_from_ads_p5,
 _flugels_inventoried_in_plant_a_in_p1,
 _flugels_inventoried_in_plant_a_in_p2,
 _flugels_inventoried_in_plant_a_in_p3,
 _flugels_inventoried_in_plant_a_in_p4,
 _flugels_inventoried_in_plant_a_in_p5,
 _flugels_inventoried_in_plant_b_in_p1,
 _flugels_inventoried_in_plant_b_in_p2,
 _flugels_inventoried_in_plant_b_in_p3,
 _flugels_inventoried_in_plant_b_in_p4,
 _flugels_inventoried_in_plant_b_in_p5,
 _flugels_produced_in_plant_a_in_p1,
 _flugels_produced_in_plant_a_in_p2,
 _flugels_produced_in_plant_a_in_p3,
 _flugels_produced_in_plant_a_in_p4,
 _flugels

In [132]:
for v in nu_industries.variables():
    print(v.name)

_flugels_advertising_investment_p1
_flugels_advertising_investment_p2
_flugels_advertising_investment_p3
_flugels_advertising_investment_p4
_flugels_contracted_demand_in_p1
_flugels_contracted_demand_in_p2
_flugels_contracted_demand_in_p3
_flugels_contracted_demand_in_p4
_flugels_contracted_demand_in_p5
_flugels_demand_from_ads_p2
_flugels_demand_from_ads_p3
_flugels_demand_from_ads_p4
_flugels_demand_from_ads_p5
_flugels_inventoried_in_plant_a_in_p1
_flugels_inventoried_in_plant_a_in_p2
_flugels_inventoried_in_plant_a_in_p3
_flugels_inventoried_in_plant_a_in_p4
_flugels_inventoried_in_plant_a_in_p5
_flugels_inventoried_in_plant_b_in_p1
_flugels_inventoried_in_plant_b_in_p2
_flugels_inventoried_in_plant_b_in_p3
_flugels_inventoried_in_plant_b_in_p4
_flugels_inventoried_in_plant_b_in_p5
_flugels_produced_in_plant_a_in_p1
_flugels_produced_in_plant_a_in_p2
_flugels_produced_in_plant_a_in_p3
_flugels_produced_in_plant_a_in_p4
_flugels_produced_in_plant_a_in_p5
_flugels_produced_in_plant_b